# stk-vis and stk

by: Andrew Tarzia

### Update 25/02/24

Installation of py3Dmol using `pip install py3Dmol` is required.

This likely will not work without some significant changes. For databasing, check out `atomlite` https://atomlite.readthedocs.io/en/latest/index.html

# Imports 

In [1]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

from rdkit import Chem 
from rdkit.Chem import AllChem as rdkit
from collections import defaultdict
from rdkit.Chem import rdFMCS
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdDistGeom
IPythonConsole.ipython_3d = True

import py3Dmol
from IPython.display import Image
import matplotlib.pyplot as plt
import subprocess
import time
import stk
import stko
import spindry as spd
import pymongo
%matplotlib inline

In [2]:
def show_stk_mol(stk_mol):
    data = rdkit.MolToMolBlock(stk_mol.to_rdkit_mol())
    p = py3Dmol.view(
        data=data,
        style={'stick':{'colorscheme':'cyanCarbon'}}, 
        width=400,
        height=400,
    )
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    p.show()

# Working

In [3]:
client = pymongo.MongoClient()

In [4]:
num_atoms_db = stk.ValueMongoDb(client, 'Num Atoms')
energy_db = stk.ValueMongoDb(client, 'UFF Energy')

ServerSelectionTimeoutError: localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 65dba69556afd46219792771, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

In [ ]:
# Create a database for depositing constructed molecules.
constructed_db = stk.ConstructedMoleculeMongoDb(
    mongo_client=client,
    # All of the parameters below are optional!
    database='stk',
    molecule_collection='molecules',
    constructed_molecule_collection='constructed_molecules',
    position_matrix_collection='position_matrices',
    building_block_position_matrix_collection='building_block_position_matrices',
)

In [ ]:
def uff_energy(molecule):
    rdkit_molecule = molecule.to_rdkit_mol()
    rdkit.SanitizeMol(rdkit_molecule)
    ff = rdkit.UFFGetMoleculeForceField(rdkit_molecule)
    return ff.CalcEnergy()

In [ ]:
# Create a constructed molecule, in this case a polymer.
polymer = stk.ConstructedMolecule(
    topology_graph=stk.polymer.Linear(
        building_blocks=(
            stk.BuildingBlock('BrC=CBr', [stk.BromoFactory()]),
            stk.BuildingBlock('BrCNCBr', [stk.BromoFactory()]),
        ),
        repeating_unit='AB',
        num_repeating_units=2,
    ),
)

# Deposit into the database.
constructed_db.put(polymer)

# You can deposit values same as before.
num_atoms_db.put(polymer, polymer.get_num_atoms())
energy_db.put(polymer, uff_energy(polymer))

In [9]:
bb1 = stk.BuildingBlock('NCCN', [stk.PrimaryAminoFactory()])
bb2 = stk.BuildingBlock(
    smiles='C1=CC(=C(C=C1Br)CBr)Br',
    functional_groups=[stk.BromoFactory()],
)
cage1 = stk.ConstructedMolecule(
    topology_graph=stk.cage.FourPlusSix(
        building_blocks=(bb1, bb2),
        optimizer=stk.MCHammer(),
    ),
)
# Deposit into the database.
constructed_db.put(cage1)

# You can deposit values same as before.
num_atoms_db.put(cage1, cage1.get_num_atoms())
energy_db.put(cage1, uff_energy(cage1))